In [ ]:
import ast
import pandas as pd
from typing import Union
from fastapi import FastAPI
from datetime import datetime
import json

### Importar el archivo ##############################################
df = pd.read_csv("movies_dataset.csv")


In [ ]:
####################### DESANIDADO #####################################

### Desanidado de "belong_to_collection"####################################

# 'belongs_to_collection' a una lista de diccionarios
df['belongs_to_collection'] = df['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None)


df['collection_id'] = df['belongs_to_collection'].apply(
    lambda x: x['id'] if isinstance(x, dict) else None)
df['collection_name'] = df['belongs_to_collection'].apply(
    lambda x: x['name'] if isinstance(x, dict) else None)
df['poster_path'] = df['belongs_to_collection'].apply(
    lambda x: x['poster_path'] if isinstance(x, dict) else None)
df['backdrop_path'] = df['belongs_to_collection'].apply(
    lambda x: x['backdrop_path'] if isinstance(x, dict) else None)


In [ ]:

### Desanidado "production_countries" ###################################

# 'production_companies' a una lista de diccionarios
df['production_companies'] = df['production_companies'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else None)


df['company_names'] = df['production_companies'].apply(
    lambda x: [company['name'] for company in x] if isinstance(x, list) else [])
df['company_ids'] = df['production_companies'].apply(
    lambda x: [company['id'] for company in x] if isinstance(x, list) else [])


In [ ]:
### Desanidado "genres" #################################

df['genre_names'] = df['genres'].apply(
    lambda x: [genre['name'] for genre in eval(x)])


In [ ]:
### Desanidado "production_countries" ####################################

#'production_countries' a una lista de diccionarios
df['production_countries'] = df['production_countries'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Desanidado
df['production_countries'] = df['production_countries'].apply(
    lambda x: [country['name'] for country in x] if isinstance(x, list) else [])


In [ ]:
### Transformacion columnas revenue y budget ##############################

df['budget'] = pd.to_numeric(
    df['budget'], errors='coerce').fillna(0).astype(int)
df['revenue'] = pd.to_numeric(
    df['revenue'], errors='coerce').fillna(0).astype(int)


# Convercion de las columnas "budget"  y "revenue" a un tipo de datos numérico

df['budget'] = df['budget'].astype(int)
df['revenue'] = df['revenue'].astype(int)





In [ ]:
### Transoformacion de la columna release_date"##############################################

# "release_date" al formato de fecha
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Formato AAAA-mm-dd 
df['release_date'] = df['release_date'].dt.strftime('%Y-%m-%d')  # 2

# Columna "release_year" 
df['release_year'] = df['release_date'].str[:4]  # 2

# Eliminar valores nulos en "release_date"
df.dropna(subset=['release_date'], inplace=True)




In [ ]:
### Columna "return"##########################################

# Calculo del retorno de inversión solo cuando revenue y budget son diferentes de 0

df['return'] = df.apply(lambda row: row['revenue'] /
                        row['budget'] if row['budget'] != 0 else 0, axis=1)


In [ ]:
### Eliminación de columnas no utilizadas y/o desanidadas ##############################

df = df.drop(columns=["belongs_to_collection", "genres","video", "imdb_id", "adult",
             "original_title", "vote_count", "poster_path", "homepage", 'id', 'overview', 'production_companies',
                      'spoken_languages', 'tagline', 'collection_id', 'poster_path', 'backdrop_path'])


In [ ]:
#df.to_csv('dataset_Transformado.csv', sep=';', index=False)


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45376 non-null  int32  
 1   original_language     45365 non-null  object 
 2   popularity            45376 non-null  object 
 3   production_countries  45376 non-null  object 
 4   release_date          45376 non-null  object 
 5   revenue               45376 non-null  int32  
 6   runtime               45130 non-null  float64
 7   status                45296 non-null  object 
 8   title                 45376 non-null  object 
 9   vote_average          45376 non-null  float64
 10  collection_name       4488 non-null   object 
 11  company_names         45376 non-null  object 
 12  company_ids           45376 non-null  object 
 13  genre_names           45376 non-null  object 
 14  release_year          45376 non-null  object 
 15  return                45